In [44]:
# In parts one and two - we extracted names from photo captions and parsed them

# Here, in part Three - we create the social graph 


# This project will scrape a New York Social Diary website and identify divs with
# photos and photo captions. Parsing the captions will yield a social network graph 
# of the New York Social Elite.


In [45]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']
import requests
from bs4 import BeautifulSoup
import urllib2
import re
from xml.sax.saxutils import escape
from HTMLParser import HTMLParser
import numpy as np
from multiprocessing import Pool
import networkx as nx
import pickle
import dill
import time
import itertools
from itertools import chain
import csv
#import nltk
#from nltk.tag import pos_tag
import matplotlib.pyplot as plt

In [46]:
# read in the list of names

list_of_words2 = pickle.load(open('all_names_for_graph_full.pickle', 'rb'))

# here we can also open the smaller pickle file

#list_of_words2 = pickle.load(open('NOV16_az_captions_parsed_small.pickle', 'rb'))


In [47]:
data_combo = []
for line in list_of_words2:
 
    try:
        if str(line) == str(['']):
            pass #print len(line) #pass
        else:
            if len(line) == 1:
                pass
            else:
                #print "yes", line
                k=(list(itertools.combinations(line,2)))    #creating all possible two tuple combinations 
                                                            #of names that are in a picture 
                data_combo.append(k)
                # added a statement not to append if one name only
                # think I did this twice then! data vs list_of_words
    
    except:   
        pass

In [48]:
data_combo[0][1]

('Les Lieberman', 'Isabel Kallman')

In [49]:
len(data_combo) # now excludes single names

67685

In [50]:
# Create graph
import operator
eM = nx.MultiGraph()
Gee = nx.Graph()


for line in data_combo:
    for node in line:
        try:                  
                eM.add_node(node[0])
                eM.add_node(node[1])
                eM.add_edge(node[0], node[1], weight = 1)
                
        except Exception, e:
            print str(e)
            pass

In [51]:
# calculate best friends - or the edges with highest weights
w=0
name1=[]
name2=[]
wt=[]
for u,v,data in eM.edges_iter(data=True):
    w = data['weight']
    if Gee.has_edge(u,v):
        Gee[u][v]['weight'] += w
    else:
        Gee.add_edge(u, v, weight=w)
        
for line in Gee.edges(data=True):
    if (str(line[0])!='') and (str(line[0])!=''):        
        name1.append(str(line[0]))
        name2.append(str(line[1]))
        k=line[2].get('weight')
        wt.append(k)

In [52]:
edges_mapped=zip(zip(name1,name2),wt)

In [53]:
edges_mapped[0:10]

[(('Christie Fraser', 'Liz Celeste'), 1),
 (('Dennis Serras', 'Marlo Bakst'), 1),
 (('Dennis Serras', 'Mikhail Baryshnikov'), 1),
 (('Dennis Serras', 'Tara Nasta'), 1),
 (('Ryan Swardstrom', 'Tom Dolby'), 1),
 (('Ryan Swardstrom', 'Peter Spears'), 1),
 (('Lynne Cherry', 'Olivia Bolar'), 1),
 (('Lynne Cherry', 'Shonda Hornbeck'), 1),
 (('Leslie Kirschenbaum', 'Elise Zealand'), 1),
 (('Samuel Barnett', 'Jano Herbosch'), 1)]

In [54]:
edges_sorted=[]
edges_sorted = sorted(edges_mapped,key=lambda x:(x[1]), reverse=True)

In [55]:
top_100edges = edges_sorted[:100]

In [56]:
top_100edges[0:10] # best friends - edges with highest weights - top 100 for the grader

[(('Bonnie Comley', 'Stewart Lane'), 75),
 (('Roric Tobin', 'Geoffrey Bradfield'), 67),
 (('Andrew Saffir', 'Daniel Benedict'), 66),
 (('Jay Diamond', 'Alexandra Lebenthal'), 51),
 (('Sessa Von Richthofen', 'Richard Johnson'), 44),
 (('Elizabeth Stribling', 'Guy Robinson'), 38),
 (('Deborah Norville', 'Karl Wellner'), 35),
 (('Wilbur Ross', 'Hilary Geary Ross'), 34),
 (('Fernanda Kellogg', 'Kirk Henckels'), 32),
 (('Douglas Hannant', 'Frederick Anderson'), 31)]

In [57]:
with open("best_friends_edges_and_weights_full.csv", "wb") as fin:
    writer = csv.writer(fin)
    writer.writerows(edges_sorted)

In [58]:
# calcualte the most popular people - top N people and their degree
nods=[]
degs=[]
for line in eM.nodes():
    nods.append(str(line))
    degs.append(eM.degree(line))
ans=zip(nods,degs)

final1 = ans
final1 = filter(None, final1)

popnames=[]
mostpop=[]
for line in final1:        
    popnames.append(line)
        
mostpop= sorted(popnames,key=lambda x:(x[1]), reverse=True)
print mostpop[1:20]

with open("most_popular_degree_full.csv", "wb") as fin:
    writer = csv.writer(fin)
    writer.writerows(mostpop)
    
# remove the cases with ['']

[('Jean Shafiroff', 603), ('Mark Gilbertson', 485), ('Gillian Miniter', 421), ('Geoffrey Bradfield', 376), ('Alexandra Lebenthal', 356), ('Gala Co', 320), ('Andrew Saffir', 293), ('Debbie Bancroft', 292), ('Patrick Mc', 284), ('Somers Farkas', 284), ('Sharon Bush', 281), ('Yaz Hernandez', 268), ('Alina Cho', 257), ('Michael Bloomberg', 255), ('Muffie Potter Aston', 250), ('Bonnie Comley', 236), ('Lydia Fenet', 234), ('Eleanora Kennedy', 233), ('Kamie Lightburn', 233)]


In [59]:
# top 100 for the grader answer
top100=[]
top100 = sorted(popnames,key=lambda x:(x[1]), reverse=True)[:100]
#print top100

In [60]:
# calculate page rank
# Use 0.85 as the damping parameter so that there is a 15% chance of jumping to
# another vertex at random. This is the nx default

pgrank=[]
pgsorted=[]
pgrank=nx.pagerank(Gee)
pgsorted = sorted(pgrank.items(), key=operator.itemgetter(1), reverse=True)
print pgsorted[1:10]
with open("pagerank_full.csv", "wb") as final:
    writer = csv.writer(final)
    writer.writerows(pgsorted)

[('Jean Shafiroff', 0.0006864700052017189), ('Mark Gilbertson', 0.0005417128682241713), ('Gillian Miniter', 0.00044507759230796146), ('Gala Co', 0.00042745248616805453), ('Geoffrey Bradfield', 0.0004116957319992117), ('Patrick Mc', 0.00040923815572880645), ('Alexandra Lebenthal', 0.00037115355512113725), ('Andrew Saffir', 0.0003613528617321991), ('Sharon Bush', 0.000322713985758053)]


In [61]:
# Save the graph to a graph file so we can operate on it with Gephi and other viz tools in a diff. notebook

In [20]:
nx.write_gexf(Gee,'Gee.gexf') # 20MB file

In [23]:
# but also write out a gml graph that can be read in by part 4.
nx.write_gml(Gee, "Gee_full.gml.gz")
nx.write_graphml(Gee, "Gee.graphml")